In [1]:
from aux_fun import get_tokens_inst_iterate_over_files

In [ ]:
# Select the instruments to keep by setting the value to True
instrument_to_keep = {
        "distorted0": True,
        "distorted1": False,
        "distorted2": False,
        "clean0": True,
        "clean1": False,
        "bass": False,
        "leads": False,    
        "pads": False,
        "drums": False,
    }

path_to_general_read_folder = "..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\data\BGTG\BGTG-v0.1"

get_tokens_inst_iterate_over_files(instrument_to_keep, path_to_general_read_folder, path_to_general_write_folder)

# 8min10s to generate all the token files with only bass and clean0
# 5min30s or 4min49s to generate all the token files with only bass (depending on whether we write the tracks that don't have bass or not)
# 8min12s to generate the token files with the guitars (clean and distorted)

Scores computed: 100%|██████████| 4871/4871 [08:12<00:00,  9.89it/s]
